### 진행 순서
1. 데이터베이스 생성
2. 상품 리스트 테이블 생성
3. 주문 목록 추가
4. 총 구매 가격 산출

### 데이터베이스 생성

In [13]:
import sqlite3

# 쇼핑몰 데이터베이스
conn = sqlite3.connect("shop.db")

# 커서 생성
c = conn.cursor()

### csv 파일로 테이블 추가

In [14]:
import pandas as pd

product_list = pd.read_csv('product_list.csv', encoding='euc-kr')
print(product_list)

    ID   상품명      가격
0    1    모자   15000
1    2    코트  200000
2    3   티셔츠   20000
3    4  블라우스   55000
4    5   가디건   45000
5    6   청바지   50000
6    7    구두  150000
7    8    가방  170000
8   10    양말   30000
9   10    양말   30000
10  10    양말   30000
11  10    양말  130000


In [15]:
# 상품 리스트 테이블에 추가
product_list.to_sql('product_list', conn, if_exists='append', index=False)

IntegrityError: UNIQUE constraint failed: product_list.id

### 튜플(리스트)로 테이블 추가

In [ ]:
# 테이블 삭제
c.execute("drop table product_list")
conn.commit()

In [ ]:
# 추가할 상품 리스트
product_list = [
    ['모자', 10000],
    ['가방', 12000],
    ['가방', 120001],
    ['가방', 120002],
    ['가방', 120003],
    ['바지', 20000],
]

# 테이블 생성
c.execute("create table if not exists product_list (id integer primary key autoincrement, 상품명 text, 가격 integer)")

# 레코드 추가
c.executemany("insert into product_list (상품명, 가격) values (?, ?)", product_list)
conn.commit()

### 주문 목록 추가

In [ ]:
c.execute(
    "create table if not exists order_list (id integer primary key autoincrement, 상품명 text, 개수 integer, 가격 integer, 총가격 integer)")
conn.commit()

In [23]:
while True:

    sep = "-" * 20

    # 상품 목록을 표시
    print(sep + "상품 목록" + sep)

    for row in c.execute("select id, 상품명, 가격 from product_list"):
        print("상품번호 : ", row[0], " / 상품명 : ", row[1], " / 가격 : ", row[2])

    # 사용자 상품 선택
    print('')
    num = input("구매하실 상품의 번호를 입력해 주세요 : ")
    c.execute("select 상품명, 가격 from product_list where id = ?", (num,))
    result = c.fetchone()

    # 상품 번호와 주문 수량을 입력
    print('')
    count = int(input("구매할 수량을 입력해 주세요 : "))
    total = count * int(result[1])

    # 주문 데이터를 db에 추가
    c.execute("insert into order_list (상품명, 개수, 가격, 총가격) values (?, ?, ?, ?)", (result[0], count, result[1], total))
    conn.commit()

    # 현재까지 주문 내역을 출력
    print('')
    print("현재까지 구매한 내역 보기")
    print(sep + "주문 목록" + sep)

    for row in c.execute("select * from order_list"):
        print("상품명 : ", row[1], " / 주문수량 : ", row[2], " / 단가 : ", row[3], " / 총가격 : ", row[4])
    print(sep + sep)

    # 사용자 추가 구매 여부
    print('')
    print("상품을 추가 구매하시겠습니까?")
    print("중단하려면 'x'를 눌러주세요.")
    print("계속 하시려면 엔터키를 눌러주세요.")

    if input("상품을 추가 구매하시겠습니까?\n중단하시려면 x를 눌러주세요.") == 'x': break

--------------------상품 목록--------------------
상품번호 :  1  / 상품명 :  모자  / 가격 :  10000
상품번호 :  2  / 상품명 :  가방  / 가격 :  12000
상품번호 :  3  / 상품명 :  가방  / 가격 :  120001
상품번호 :  4  / 상품명 :  가방  / 가격 :  120002
상품번호 :  5  / 상품명 :  가방  / 가격 :  120003
상품번호 :  6  / 상품명 :  바지  / 가격 :  20000


현재까지 구매한 내역 보기
--------------------주문 목록--------------------
상품명 :  가방  / 주문수량 :  3  / 단가 :  12000  / 총가격 :  36000
----------------------------------------

상품을 추가 구매하시겠습니까?
중단하려면 'x'를 눌러주세요.
계속 하시려면 엔터키를 눌러주세요.
--------------------상품 목록--------------------
상품번호 :  1  / 상품명 :  모자  / 가격 :  10000
상품번호 :  2  / 상품명 :  가방  / 가격 :  12000
상품번호 :  3  / 상품명 :  가방  / 가격 :  120001
상품번호 :  4  / 상품명 :  가방  / 가격 :  120002
상품번호 :  5  / 상품명 :  가방  / 가격 :  120003
상품번호 :  6  / 상품명 :  바지  / 가격 :  20000


현재까지 구매한 내역 보기
--------------------주문 목록--------------------
상품명 :  가방  / 주문수량 :  3  / 단가 :  12000  / 총가격 :  36000
상품명 :  가방  / 주문수량 :  2  / 단가 :  12000  / 총가격 :  24000
----------------------------------------

상품을 추가 구매하시겠습니까?
중단

### 총 구매 가격 산출

In [24]:
# 주문 내역에서 총 구매 가격 계산
print("총 구매가격", end="")

for row in c.execute("select sum(총가격) from order_list"):
    print(" : ", row[0], "원")

print('')

# 주문 내역 초기화
c.execute("delete from order_list")
conn.commit()
# 데이터베이스 연결 해제
# conn.close()

총 구매가격 :  60000 원
